<a href="https://colab.research.google.com/github/kalindasiaminwe/Designing-a-Bank-Marketing-Database/blob/main/Designing_a_Bank_Marketing_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="https://assets.datacamp.com/production/repositories/6005/datasets/0c64652120b80cfe2762012d7252b439138be223/piggy_bank.jpg" alt="Piggy bank" width="600" height="500"></center>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf)!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll provide the database design script as a `.sql` file that they can then run. 

You have been asked to design a database that will have three tables:

## `client`
| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `text` | Whether the client's credit is in default |
| `housing` | `text` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `text` | Whether the client has an existing personal loan |

## `campaign`
| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `integer` | Campaign ID |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `text` | Outcome of the previous campaign |
| `campaign_outcome` | `integer` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

## `economics`
| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

We will start off by importing pandas and numpy 

In [ ]:
#Import libraries
import pandas as pd
import numpy as np

Once thats done, we load our csv file as a dataframe and explore it before we begin transforming it to the requirements given to us by the bank.

In [ ]:
data = pd.read_csv("/content/bank_marketing.csv") 

In [ ]:
#Check columns contained in the dataset
data.columns

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration',
       'campaign', 'pdays', 'previous', 'poutcome', 'emp_var_rate',
       'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'y'],
      dtype='object')

In [ ]:
#Get an overview of the dataset
data.info

<bound method DataFrame.info of        client_id  age          job  ... euribor3m nr_employed    y
0              0   56    housemaid  ...     4.857      5191.0   no
1              1   57     services  ...     4.857      5191.0   no
2              2   37     services  ...     4.857      5191.0   no
3              3   40       admin.  ...     4.857      5191.0   no
4              4   56     services  ...     4.857      5191.0   no
...          ...  ...          ...  ...       ...         ...  ...
41183      41183   73      retired  ...     1.028      4963.6  yes
41184      41184   46  blue-collar  ...     1.028      4963.6   no
41185      41185   56      retired  ...     1.028      4963.6   no
41186      41186   44   technician  ...     1.028      4963.6  yes
41187      41187   74      retired  ...     1.028      4963.6   no

[41188 rows x 22 columns]>

In [ ]:
#Get an overview of the data in the dataset and take note of the things that the Bank wants changed, added or deleted.
data.head()

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


Now that we know what the dataset looks like, we can proceed to transforming the dataset. 

First things first, the bank database is to have three tables: client, campaign and economic. Therefore, we will split the *bank_marketing.csv* into three with the respective columns.

In [ ]:
client = data[["client_id", "age", "job", "marital", "education", "credit_default", "housing", "loan"]]

campaign = data[["client_id", "campaign", "month", "day_of_week", 
               "duration", "pdays", "previous", "poutcome", "y"]]

economics = data[["client_id", "emp_var_rate", "cons_price_idx", 
                "euribor3m", "nr_employed"]]

Then we will rename columns as follows: 

*  "client_id" to "id" in client. *(leave as-is in the other subsets)*;
*    "duration" to "contact_duration", "previous" to "previous_campaign_contacts", "y" to "campaign_outcome", "poutcome" to "previous_outcome", and "campaign" to "number_contacts" in campaign;
* "euribor3m" to "euribor_three_months" and "nr_employed" to "number_employed" in economics.

In [ ]:
client.rename(columns={"client_id":"id"}, inplace=True)

campaign.rename(columns={"duration":"contact_duration", "previous":"previous_campaign_contacts", "y":"campaign_outcome", "poutcome":"previous_outcome", "campaign":"number_contacts"}, inplace=True)

economics.rename(columns={"euribor3m":"euribor_three_months", "nr_employed":"number_employed"}, inplace=True)


We check the columns for each table to ensure the changes were successful.
***NOTE: Uncomment the table you want to check & ensure the other two are commented.***

In [ ]:
client.columns
# campaign.columns
# economics.columns

Index(['id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing',
       'loan'],
      dtype='object')

Next we will clean up the data in the columns. 

**For client:**
*   In the "education" column, changing "." to "_" and "unknown" to NumPy's null values.
*   Remove periods from the "job" column.

**For campaign:**

*   Convert "previous_outcome" and "campaign_outcome" values to binary (1 or 0), including the conversion of "nonexistent" to 0.



In [ ]:
client.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes


In [ ]:
client["education"] = client["education"].str.replace(".", "_")
client["education"] = client["education"].replace("unknown", np.NaN)
client["job"] = client["job"].str.replace(".", "")

In [ ]:
campaign.tail()

,campaign_id,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
41183,1,41183,1,334,999,0,0,0,nov-fri-2022
41184,1,41184,1,383,999,0,0,0,nov-fri-2022
41185,1,41185,2,189,999,0,0,0,nov-fri-2022
41186,1,41186,1,442,999,0,0,0,nov-fri-2022
41187,1,41187,3,239,999,1,0,0,nov-fri-2022


In [ ]:
campaign["campaign_outcome"] = campaign["campaign_outcome"].replace("yes", 1).replace("no", 0).replace("failure", 0)

campaign["previous_outcome"] = campaign["previous_outcome"].replace("nonexistent", 0).replace("failure", 0).replace("success", 1)

Next in the campaign, we add a campaign_id as the first column in campaign, where all rows have a value of 1.

We also create a datetime column called last_contact_date, in the format of "month-day-year", where the year is 2022, and the month and day values are taken from the "month" and "day_of_week" columns.

In [ ]:
campaign["year"] = "2022"
campaign["last_contact_date"] = campaign["month"] + "-" + campaign["day_of_week"] + "-" + campaign["year"]

campaign["campaign_id"] = 1

In [ ]:
#rearrange columns to make campaign_id first
campaign = campaign[["campaign_id",
    "client_id",
    "number_contacts",
    "month",
    "day_of_week",
    "contact_duration",
    "pdays",
    "previous_campaign_contacts",
    "previous_outcome",
    "campaign_outcome",
    "year",
    "last_contact_date"
    ]]

In [ ]:
#Ensure all changes were successful
campaign.head()

,campaign_id,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,1,0,1,261,999,0,0,0,may-mon-2022
1,1,1,1,149,999,0,0,0,may-mon-2022
2,1,2,1,226,999,0,0,0,may-mon-2022
3,1,3,1,151,999,0,0,0,may-mon-2022
4,1,4,1,307,999,0,0,0,may-mon-2022


Then we drop unnecessary columns, ensuring the columns in each subset of the data match the table displayed at the beginning of the notebook.  

In [ ]:
campaign.drop(columns=["month", "year", "day_of_week"], inplace=True)

Save the three subsets into csv files without an index

In [ ]:
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)

Lastly, we Create a dictionary called database_design using the three table names as keys and assigning values as the respective subset DataFrames you have created.

In [ ]:
# Store and print database_design
database_design = {"client": client,
                  "campaign": campaign,
                  "economics": economics}

print(database_design)

{'client':           id  age          job  ... credit_default housing loan
0          0   56    housemaid  ...             no      no   no
1          1   57     services  ...        unknown      no   no
2          2   37     services  ...             no     yes   no
3          3   40        admin  ...             no      no   no
4          4   56     services  ...             no      no  yes
...      ...  ...          ...  ...            ...     ...  ...
41183  41183   73      retired  ...             no     yes   no
41184  41184   46  blue-collar  ...             no      no   no
41185  41185   56      retired  ...             no     yes   no
41186  41186   44   technician  ...             no      no   no
41187  41187   74      retired  ...             no     yes   no

[41188 rows x 8 columns], 'campaign':        campaign_id  client_id  ...  campaign_outcome  last_contact_date
0                1          0  ...                 0       may-mon-2022
1                1          1  ...    

**Once we are done cleaning and reformatting the data, we also have to provide the bank with a .sql file that contains SQL code that the bank can execute to create the tables and populate with the data from the csv files.**

In [ ]:
#@title bank_marketing.sql
-- Create clients table
CREATE TABLE clients (
    id SERIAL PRIMARY KEY,
    age INTEGER NOT NULL,
    job TEXT NOT NULL,
    marital TEXT NOT NULL,
    education TEXT NOT NULL,
    credit_default TEXT NOT NULL,
    housing TEXT NOT NULL,
    loan TEXT NOT NULL
);

-- Populate clients table with data from clients.csv file
COPY clients FROM '/path/to/clients.csv' DELIMITER ',' CSV HEADER;

-- Create campaigns table
CREATE TABLE campaigns (
    campaign_id INTEGER PRIMARY KEY,
    client_id INTEGER NOT NULL REFERENCES clients(id),
    number_contacts INTEGER NOT NULL,
    contact_duration INTEGER NOT NULL,
    pdays INTEGER NOT NULL,
    previous_campaign_contacts INTEGER NOT NULL,
    previous_outcome TEXT NOT NULL,
    campaign_outcome INTEGER NOT NULL,
    last_contact_date DATE NOT NULL
);

-- Populate campaigns table with data from campaigns.csv file
COPY campaigns FROM '/path/to/campaigns.csv' DELIMITER ',' CSV HEADER;

-- Create economics table
CREATE TABLE economics (
    client_id INTEGER NOT NULL REFERENCES clients(id),
    emp_var_rate FLOAT NOT NULL,
    cons_price_idx FLOAT NOT NULL,
    euribor_three_months FLOAT NOT NULL,
    number_employed FLOAT NOT NULL
);

-- Populate economics table with data from economics.csv file
COPY economics FROM '/path/to/economics.csv' DELIMITER ',' CSV HEADER;


**NOTE: WE PROVIDED THE SQL CODE ABOVE IN A SEPARATE .SQL FILE.**

You can execute it using any SQL client or command line tool that supports PostgreSQL.